In [7]:
import polars as pl 
import numpy as np 

In [73]:
df = pl.read_csv('D:/WAIS.csv', try_parse_dates=True,infer_schema_length=10000)

# df = df.with_columns(pl.col("stk_ex_date").str.strptime(pl.Date, format="%Y-%m-%d"))
# df = df.with_columns(
#     df['pre_close'].cast(pl.Float64),
#     df['ex_price'].cast(pl.Float64),
#     df['issue_price'].cast(pl.Float64),
#     df['pre_ex_total_share'].cast(pl.Int64),
#     df['stkcap_total_share'].cast(pl.Int64))
df_with_year = df.with_columns(pl.col("zdate").dt.year().alias("year"))
df_with_year_month = df_with_year.with_columns(pl.col("zdate").dt.month().alias("month"))
df2 = df_with_year_month.filter(pl.col("stk_ex_date") .is_not_null())
df2 = df2.filter(pl.col("adjfac_type") =='TB')
# df2 = df2.filter(pl.col("pre_ex_total_share") .is_not_null())
# df2 = df2.filter(pl.col("issue_price") .is_not_null())
# df2 = df2.filter(pl.col("issue_price") != 0)
df_test = df2.with_columns((12-pl.col("month")).alias("M"))
# df_test = df_test.with_columns((pl.col("ex_price")/pl.col("pre_close")).alias("J"))
df_test
# df_with_year_month

tej_fp_id,stk_id,tej_comp_id,stk_type,close,zdate,stk_ex_date,pre_close,ex_price,issue_share_tej,cashback,adjfac_type,j,adj,cap_dec_share,cap_dec_share_amt,issue_price,pre_ex_total_share,stkcap_total_share,year,month,M
i64,str,i64,str,f64,date,date,f64,f64,i64,f64,str,f64,f64,i64,i64,f64,i64,i64,i32,u32,u32
53265,"""1416""",34101,"""1""",11.7,2023-06-16,2023-06-16,12.05,11.55,185342168,null,"""TB""",0.958506,1.0,null,null,null,null,null,2023,6,6
54577,"""2385""",29459,"""1""",95.2,2023-06-16,2023-06-16,105.0,97.5,760053172,null,"""TB""",0.928571,1.0,null,null,null,null,null,2023,6,6
54589,"""2388""",38486,"""1""",84.6,2023-06-16,2023-06-16,81.9,81.75,498344941,null,"""TB""",0.998168,1.0,null,null,null,null,null,2023,6,6
55365,"""2723""",32438,"""1""",150.0,2023-06-16,2023-06-16,154.0,151.5,180000000,null,"""TB""",0.983766,1.0,null,null,null,null,null,2023,6,6
56672,"""3679""",33842,"""1""",85.7,2023-06-16,2023-06-16,94.7,84.7,63024229,null,"""TB""",0.894403,1.0,null,null,null,null,null,2023,6,6
56673,"""3680""",28523,"""1""",400.5,2023-06-16,2023-06-16,406.0,402.06,86588210,null,"""TB""",0.990271,1.0,null,null,null,null,null,2023,6,6
56727,"""4128""",35744,"""1""",57.5,2023-06-16,2023-06-16,61.3,55.89,560162474,0.0,"""TB""",0.911746,1.0,0,0,0.0,511097148,560162474,2023,6,6
56902,"""4743""",33163,"""1""",216.0,2023-06-16,2023-06-16,230.5,202.51,450613425,0.0,"""TB""",0.877874,1.0,0,0,0.0,395765227,450613425,2023,6,6
56991,"""4974""",36981,"""1""",65.2,2023-06-16,2023-06-16,71.4,65.6,72500104,null,"""TB""",0.918768,1.0,null,null,null,null,null,2023,6,6


In [69]:
df = pl.read_csv('D:/WAISFIN.csv', try_parse_dates=True)
selected_columns = ['tej_comp_id', 'fin_year', 'quarter','fin_type','fin_aq','acc_code','acc_value']
df_selected = df.select(selected_columns)

transposed = df_selected.pivot( values="acc_value", index=["tej_comp_id", 'fin_year','fin_aq', 'quarter','fin_type'], columns="acc_code", aggregate_function="first")
acc_transposed = transposed.select('tej_comp_id', pl.col('fin_year').alias('zdate'),'fin_aq', 'quarter','fin_type',pl.col('211F').alias('wavg_com')
                        ,pl.col('3950').alias('netprofit_com')
                        ,pl.col('3990').alias('eps_com')
                        ,pl.col('7210').alias('cashflow_operate_com')
                        ,pl.col('240D').alias('par_Buyback_com')
                        ,pl.col('240E').alias('chi_Buyback_com')
                        ,pl.col('240G').alias('Buyback_com'))
acc_transposed = acc_transposed.with_columns('quarter',pl.when(acc_transposed['quarter'] == 4).then(12)
                                             .when(acc_transposed['quarter'] == 3).then(9)
                                             .when(acc_transposed['quarter'] == 2).then(6)
                                             .when(acc_transposed['quarter'] == 1).then(3).alias('month').cast(pl.UInt32),
                                             pl.col("zdate").dt.year().alias("year").cast(pl.Int32))
acc_transposed

tej_comp_id,zdate,fin_aq,quarter,fin_type,wavg_com,netprofit_com,eps_com,cashflow_operate_com,par_Buyback_com,chi_Buyback_com,Buyback_com,month,year
i64,date,str,i64,str,f64,f64,f64,f64,f64,f64,f64,u32,i32
25574,1975-01-01,"""A""",4,"""A""",22000.0,84717.0,3.85,null,0.0,0.0,0.0,12,1975
25830,1975-01-01,"""A""",4,"""A""",5000.0,12071.0,2.41,null,0.0,0.0,0.0,12,1975
26787,1975-01-01,"""A""",4,"""A""",20000.0,469.0,0.02,null,null,null,null,12,1975
32286,1975-01-01,"""A""",4,"""A""",10000.0,43122.0,4.31,null,0.0,0.0,0.0,12,1975
36284,1975-01-01,"""A""",4,"""A""",12500.0,11569.0,0.93,null,0.0,0.0,0.0,12,1975
25573,1976-01-01,"""A""",4,"""A""",12500.0,-2136.0,-0.17,null,0.0,0.0,0.0,12,1976
25574,1976-01-01,"""A""",4,"""A""",30000.0,80255.0,2.68,null,0.0,0.0,0.0,12,1976
25830,1976-01-01,"""A""",4,"""A""",8000.0,7672.0,0.96,null,0.0,0.0,0.0,12,1976
26352,1976-01-01,"""A""",4,"""A""",8000.0,2.22,0.28,null,0.0,0.0,0.0,12,1976


In [77]:
modual_df = df_test.join(acc_transposed , on =['tej_comp_id','month','year'] , how='left')
modual_df = modual_df.filter(pl.col('wavg_com').is_not_null())
modual_df = modual_df.with_columns(pl.col('zdate').dt.strftime('%Y%m').alias('year_month'))
modual_df = modual_df.sort(['zdate','tej_fp_id'])
modual_df

tej_fp_id,stk_id,tej_comp_id,stk_type,close,zdate,stk_ex_date,pre_close,ex_price,issue_share_tej,cashback,adjfac_type,j,adj,cap_dec_share,cap_dec_share_amt,issue_price,pre_ex_total_share,stkcap_total_share,year,month,M,zdate_right,fin_aq,quarter,fin_type,wavg_com,netprofit_com,eps_com,cashflow_operate_com,par_Buyback_com,chi_Buyback_com,Buyback_com,year_month
i64,str,i64,str,f64,date,date,f64,f64,i64,f64,str,f64,f64,i64,i64,f64,i64,i64,i32,u32,u32,date,str,i64,str,f64,f64,f64,f64,f64,f64,f64,str
53791,"""1801""",34343,"""1""",40.6,1979-12-26,1979-12-26,44.8,39.65,30510000,0.0,"""TB""",0.910491,0.489752,0,0,10.0,27000000,30510000,1979,12,0,1979-01-01,"""A""",4,"""A""",30510.0,128060.0,4.2,null,0.0,0.0,0.0,"""197912"""
58408,"""8701""",35009,"""1""",16.9,1980-12-10,1980-12-10,18.1,16.48,20000000,0.0,"""TB""",0.910497,0.29676,0,0,10.0,16000000,20000000,1980,12,0,1980-01-01,"""A""",4,"""A""",16000.0,30328.0,1.9,null,0.0,0.0,0.0,"""198012"""
54041,"""2102""",34187,"""1""",23.0,1981-12-07,1981-12-07,23.8,21.63,47850000,0.0,"""TB""",0.908824,0.095854,0,0,0.0,43500000,47850000,1981,12,0,1981-01-01,"""A""",4,"""A""",47850.0,-8374.0,-0.18,null,0.0,0.0,0.0,"""198112"""
53046,"""1203""",26360,"""1""",11.45,1981-12-10,1981-12-10,11.8,11.23,105000000,0.0,"""TB""",0.951695,0.185392,0,0,0.0,100000000,105000000,1981,12,0,1981-01-01,"""A""",4,"""A""",105000.0,-19583.0,-0.19,null,0.0,0.0,0.0,"""198112"""
54515,"""2371""",26651,"""1""",10.0,1981-12-12,1981-12-12,10.6,9.37,413750000,0.0,"""TB""",0.891509,0.204468,0,0,10.0,365000000,413750000,1981,12,0,1981-01-01,"""A""",4,"""A""",413750.0,148991.0,0.36,995659.0,0.0,0.0,0.0,"""198112"""
55577,"""2901""",26756,"""1""",20.6,1981-12-14,1981-12-14,23.3,19.47,27572000,0.0,"""TB""",0.835622,0.418258,0,0,10.0,20900000,27572000,1981,12,0,1981-01-01,"""A""",4,"""A""",20067.0,3001.0,0.15,-2228.0,0.0,0.0,0.0,"""198112"""
53245,"""1408""",33710,"""1""",9.5,1981-12-15,1981-12-15,9.35,9.35,219562200,null,"""TB""",1.0,0.558982,null,null,null,201275300,219562200,1981,12,0,1981-01-01,"""A""",4,"""A""",219562.2,46000.0,0.21,null,0.0,0.0,0.0,"""198112"""
53265,"""1416""",34101,"""1""",10.05,1981-12-18,1981-12-18,11.0,9.99,74517700,0.0,"""TB""",0.908182,0.248818,0,0,0.0,67743300,74517700,1981,12,0,1981-01-01,"""A""",4,"""A""",74517.7,-47933.0,-0.64,null,0.0,0.0,0.0,"""198112"""
53282,"""1429""",34088,"""1""",18.5,1981-12-21,1981-12-21,18.9,17.66,23540000,0.0,"""TB""",0.934392,0.608197,0,0,0.0,22000000,23540000,1981,12,0,1981-01-01,"""A""",4,"""A""",23540.0,20240.0,0.86,null,0.0,0.0,0.0,"""198112"""


In [67]:
result = modual_df.groupby(["tej_fp_id","year"]).agg(
    (pl.col("pre_ex_total_share")/pl.col("j")*(12-pl.col("M"))/12+pl.col("stkcap_total_share")*pl.col("M")/12).alias("AVG").cast(pl.Int64))
df_test = df_test.join(result, on =  ['tej_fp_id' , 'year' ], how ='left')
df_test 
result

tej_fp_id,year,AVG
i64,i32,list[i64]
53187,2023,"[1356567732, null]"
53433,2023,[300080376]
53489,2023,[144031625]
53555,2023,[37485537]
54400,2023,[560156397]
54627,2023,[778435271]
55003,2023,[736777382]
55385,2023,[102193220]
55729,2023,"[239035022, 238971891]"


In [5]:
df_test = df_test.filter(pl.col('tej_fp_id')==54670)
df_test = df_test.with_columns((12-pl.col("month")).alias("M"))
df_test = df_test.with_columns((pl.col("ex_price")/pl.col("pre_close")).alias("J"))
df_test = df_test.with_columns((pl.col("pre_ex_total_share")/pl.col("J")*(12-pl.col("M"))/12+pl.col("stkcap_total_share")*pl.col("M")/12).alias("AVG").cast(pl.Int64))

df_test

tej_fp_id,stk_type,stk_id,close,zdate,stk_ex_date,pre_close,ex_price,issue_price,pre_ex_total_share,stkcap_total_share,year,month,M,J,AVG
i64,i64,str,f64,date,date,f64,f64,f64,i64,i64,i32,u32,u32,f64,i64
54670,1,"""2409""",25.3,2001-01-17,2001-01-17,24.9,24.55,22.0,1100000000,1250000000,2001,1,11,0.985944,1238806856
54670,1,"""2409""",41.6,2002-05-23,2002-05-23,41.7,41.7,39.72,2970581607,3470581607,2002,5,7,1.0,3262248273
54670,1,"""2409""",51.0,2004-06-18,2004-06-18,53.0,53.0,53.8,4658040897,4958040897,2004,6,6,1.0,4808040897
54670,1,"""2409""",50.6,2005-07-22,2005-07-22,51.1,51.1,48.9,5500547132,5830547132,2005,7,5,1.0,5638047132
54670,1,"""2409""",13.45,2013-05-07,2013-05-07,13.65,13.65,13.04,8827045535,9624245115,2013,5,7,1.0,9292078623
